In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import time 

import numpy as np
import pinocchio as pin
from tqdm import tqdm

from projectyl.dynamics.armmodel import ArmRobot
from projectyl.dynamics.meshcat_viewer_wrapper import MeshcatVisualizer

Let's create a robot arm

In [ ]:
arm_robot = ArmRobot(upper_arm_length=0.3, forearm_length=0.25)

In [ ]:
viz = MeshcatVisualizer(arm_robot)
viz.display(arm_robot.q0)

In [ ]:
# Iterate through all the joints in the robot's model
for i in range(arm_robot.model.njoints):
    joint_name = arm_robot.model.names[i]
    joint_model = arm_robot.model.joints[i]

    joint_type = joint_model.shortname()

    parent_joint_id = arm_robot.model.parents[i]
    parent_joint_name = arm_robot.model.names[parent_joint_id]

    # Print joint information
    print("Joint ID:", i)
    print("Joint Name:", joint_name)
    print("Joint Type:", joint_type)
    print("Parent Joint ID:", parent_joint_id)
    print("Parent Joint Name:", parent_joint_name)
    print()

In [ ]:
# Iterate through all the frames in the robot's model
for i in range(len(arm_robot.model.frames)):
    frame = arm_robot.model.frames[i]
    parent_joint_id = frame.parent
    parent_joint_name = arm_robot.model.names[parent_joint_id]
    print("Frame ID:", i)
    print("Frame Name:", frame.name)
    print("Frame Type:", frame.type)
    print("Parent Joint ID:", parent_joint_id)
    print("Parent Joint name:", parent_joint_name)
    print()

In [ ]:
print("Initial configuration:", arm_robot.q0)
print("Config dim:", arm_robot.nq)
print("DoF:", arm_robot.nv)

Test : Random configuration

In [ ]:
for _ in range(100):
    q = pin.randomConfiguration(arm_robot.model)
    viz.display(q)
    time.sleep(1e-2)

Test: Forward Kinematics

In [ ]:
q = pin.randomConfiguration(arm_robot.model)
pin.framesForwardKinematics(arm_robot.model, arm_robot.data, q)

viz.display(q)

In [ ]:
for i in range(arm_robot.model.njoints):
    joint_name = arm_robot.model.names[i]
    joint_position = arm_robot.data.oMi[i]
    print("Joint Name:", joint_name)
    print("Joint Position in World Frame:", joint_position)
    print()


In [ ]:
for i in range(len(arm_robot.model.frames)):
    frame = arm_robot.model.frames[i]
    frame_placement = arm_robot.data.oMf[i]
    print("Frame Name:", frame.name)
    print("Frame Position in World Frame:", frame_placement)
    print()

In [ ]:
# Test: confirm the position of the end effector relative to the elbow
elbow_id = arm_robot.model.getFrameId("elbow")
wrist_id = arm_robot.model.getFrameId("wrist")

elbow_transform = arm_robot.data.oMf[elbow_id]
wrist_transform = arm_robot.data.oMf[wrist_id]

relative_transform = elbow_transform.inverse() * wrist_transform
print("Relative position of the end effector with respect to the elbow, computed:", relative_transform)
print("Real value:", arm_robot.model.frames[wrist_id].placement)

Test: Inverse Kinematics

In [ ]:
# Create first a target
q = pin.randomConfiguration(arm_robot.model)
pin.framesForwardKinematics(arm_robot.model, arm_robot.data, q)

target_position_end_effector = arm_robot.data.oMf[wrist_id].copy()

In [ ]:
# Print the target
viz.addBox("goal", [.1, .1, .1], [.1, .1, .5, .6])
viz.applyConfiguration("goal", target_position_end_effector)

In [ ]:
# Robot initial position
q0 = arm_robot.q0
DT = 1e-2
viz.display(q0)

In [ ]:
q = q0.copy()

for i in tqdm(range(500)):

    # end effector frame
    pin.framesForwardKinematics(arm_robot.model, arm_robot.data, q)
    oMeffector = arm_robot.data.oMf[wrist_id]

    # Displacement between F_effector and F_goal
    effectorMgoal = oMeffector.inverse() * target_position_end_effector

    # error between the two
    effector_nu = pin.log(effectorMgoal).vector

    # Jacobian in the effector frame
    effector_Jeffector = pin.computeFrameJacobian(arm_robot.model, arm_robot.data, q, wrist_id, pin.LOCAL)

    # Control law
    vq = np.linalg.pinv(effector_Jeffector) @ effector_nu

    # Integrate
    q = pin.integrate(arm_robot.model, q, vq*DT)

    viz.display(q)
    time.sleep(1e-3)

In [ ]:
# remove target
viz.delete("goal")

Test: Forward dynamics

In [ ]:
# Random position, speed and torques of the robot
q = pin.randomConfiguration(arm_robot.model)
vq = np.random.rand(arm_robot.model.nv)
tauq = np.random.rand(arm_robot.model.nv)

print("Initial joint position:", q)
print("Initial joint velocity:", vq)
print("Initial torques")


b = pin.nle(arm_robot.model, arm_robot.data, q, vq)
M = pin.crba(arm_robot.model, arm_robot.data, q)
print("Mass matrix of the robot:\n", M)
print("Nonlinear effects", b)

aq = np.linalg.solve(M, tauq - b)
print("Accelleration", aq)

print("Tests (should be 0)", 
    np.linalg.norm(pin.rnea(arm_robot.model, arm_robot.data, q, vq, aq) - tauq),
    np.linalg.norm(pin.rnea(arm_robot.model, arm_robot.data, q, vq, np.zeros(arm_robot.model.nv)) - b)  
)

In [ ]:
# Robot initial position
q0 = arm_robot.q0.copy()
q0[4:] = pin.randomConfiguration(arm_robot.model)[4:] # For more fun
DT = 1e-2
Kf = 0.1 # friction
viz.display(q0)

In [ ]:
# Free fall 
q = q0.copy()
vq = np.zeros(arm_robot.model.nv)
tauq = np.zeros(arm_robot.model.nv)

for i in tqdm(range(1000)):

    # Compute mass and non linear effects
    M = pin.crba(arm_robot.model, arm_robot.data, q)
    b = pin.nle(arm_robot.model, arm_robot.data, q, vq)

    # Compute accelleration
    aq = np.linalg.solve(M, tauq - b)

    # Integrate acceleration and speed
    vq += aq * DT
    q = pin.integrate(arm_robot.model, q, vq * DT)

    # Update torque (friction)
    tauq = - Kf * vq


    viz.display(q)
    time.sleep(1e-3)